# Final Project - Reinforcements Learning 
Hello dear students,<br> this is the template notebook. Please click on the "File" tab and then on "Save a copy into drive".

---
<br>

### Name and ID:
Student 1: Avraham Raviv, 204355390
<br>
Student 2: Yevgeni Berkovitch, 317079234
<br><br>
<img src="https://play-lh.googleusercontent.com/e_oKlKPISbgdzut1H9opevS7-LTB8-8lsmpCdMkhlnqFenZhpjxbLmx7l158-xQQCIY">

### https://github.com/mpSchrader/gym-sokoban

# Installs

In [1]:
%%capture
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install gym
!pip install pygame
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym_sokoban

!imageio_download_bin ffmpeg

# Imports

In [2]:
import random
import time

import numpy as np
import scipy as scp
import matplotlib.pyplot as plt

import base64
import imageio
from pyvirtualdisplay import Display
from IPython.display import HTML

import gym
from gym import error, spaces, utils
from soko_pap import *

from collections import deque
from queue import PriorityQueue

from keras.models import Sequential
from keras.layers import Dense

from tqdm.notebook import tqdm
from collections import defaultdict

In [3]:
%matplotlib inline

In [4]:
imageio.plugins.ffmpeg.download()

In [5]:
from gym import logger as gymlogger
gymlogger.set_level(40) # error only

# Display utils
The cell below contains the video display configuration. No need to make changes here.

In [6]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return HTML(tag)

# Utils

In [7]:
def get_distances(room_state):
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):
            if room_state[i][j] == 2:
                target = (i, j)

    distances = np.zeros(shape=room_state.shape)
    visited_cells = set()
    cell_queue = deque()

    visited_cells.add(target)
    cell_queue.appendleft(target)

    while len(cell_queue) != 0:
        cell = cell_queue.pop()
        distance = distances[cell[0]][cell[1]]
        for x,y in ((1,0), (-1,-0), (0,1), (0,-1)):
            next_cell_x, next_cell_y = cell[0]+x, cell[1]+y
            if room_state[next_cell_x][next_cell_y] != 0 and not (next_cell_x, next_cell_y) in visited_cells:
                distances[next_cell_x][next_cell_y] = distance + 1
                visited_cells.add((next_cell_x, next_cell_y))
                cell_queue.appendleft((next_cell_x, next_cell_y))
                
    return distances   

def calc_distances(room_state, distances):
    box = None
    mover = None
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):            
            if room_state[i][j] == 4:
                box = (i,j)
            
            if room_state[i][j] == 5:
                mover = (i,j)
    
    return mover, box, distances[box[0]][box[1]]   

def box2target_change_reward(room_state, next_room_state, distances):
    if np.array_equal(room_state, next_room_state):
        return -1.0
    
    mover, box, t2b = calc_distances(room_state, distances)
    n_mover, n_box, n_t2b = calc_distances(next_room_state, distances)
    
    change_reward = 0.0
    if n_t2b < t2b:
        change_reward += 5.0
    elif n_t2b > t2b:
        change_reward -= 5.0
        
    m2b = np.sqrt((mover[0]-box[0])**2 + (mover[1]-box[1])**2)
    n_m2b = np.sqrt((n_mover[0]-n_box[0])**2 + (n_mover[1]-n_box[1])**2)
    
    if n_m2b < m2b and m2b >= 2:
        change_reward += 1.0
    elif n_m2b > m2b and n_m2b >= 2:
        change_reward -= 1.0
        
    return change_reward   

# Solution

In [8]:
class SOK_Agent:
    def __init__(self):
        # Construct DQN models
        self.state_size = (25,) 
        self.action_size = 8
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.batch_size = 8
        
        # Replay buffers
        self.replay_buffer = deque(maxlen=5000)
        self.prioritized_replay_buffer = deque(maxlen=500)
        
        # Hyperparameters
        self.gamma = 0.9
        self.epsilon = 1.0   
        self.epsilon_min = 0.3
        self.epsilon_decay = 0.995
        self.replay_rate = 10
        self.update_beta = 0.999

        self.verbosity = 100 

    def _build_model(self):
        model = Sequential()
        model.add(Dense(25, input_shape=self.state_size, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer="adam")        
        return model

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append([state, action, reward, next_state, done])    
        
    def copy_to_prioritized_buffer(self, n):
        for i in range(n):
            self.prioritized_replay_buffer.append(self.replay_buffer[-1-i])  

    def act(self, state, stochastic=False):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state, verbose=0)[0]
        
        if stochastic:
            act_probs = np.exp(act_values)/np.exp(act_values).sum()
            return np.random.choice(np.arange(self.action_size), size=1, p=act_probs)[0]
              
        return np.argmax(act_values) 

    def replay(self): 
        if len(self.replay_buffer) < self.batch_size:
            return
        
        if len(self.prioritized_replay_buffer) < self.batch_size//2:
            minibatch = random.sample(self.replay_buffer, self.batch_size) 
        else:    
            minibatch = random.sample(self.replay_buffer, self.batch_size//2) 
            minibatch.extend(random.sample(self.prioritized_replay_buffer, self.batch_size//2))
        
        states = np.zeros((self.batch_size, self.state_size[0]))
        actions = np.zeros(self.batch_size, dtype=int)
        rewards = np.zeros(self.batch_size)
        next_states = np.zeros((self.batch_size, self.state_size[0]))
        statuses = np.zeros(self.batch_size)
        targets = np.zeros((self.batch_size, self.action_size)) 
        
        for i, (state, action, reward, next_state, done) in enumerate(minibatch): 
            states[i] = state.copy()
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state.copy()
            statuses[i] = 1 if done else 0    
        
        targets = self.model.predict(states) 
        max_actions = np.argmax(self.model.predict(next_states), axis=1)
        next_rewards = self.target_model.predict(next_states)
        
        ind = 0
        for action, reward, next_reward, max_action, done in zip(actions, rewards, next_rewards, max_actions, statuses):  
            if not done:
                reward += self.gamma * next_reward[max_action]
            targets[ind][action] = reward
            ind += 1
        
        self.model.fit(states, targets, epochs=10, verbose=0) 
        
        self.update_target_model()        
    
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
        
    def update_target_model(self):
        model_w = self.model.get_weights()
        target_model_w = self.target_model.get_weights()
        updated_target_model_w = []
        for i in range(len(model_w)):
            updated_target_model_w.append(self.update_beta*target_model_w[i] + (1-self.update_beta)*model_w[i])
        self.target_model.set_weights(updated_target_model_w)    
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [9]:
def process_frame(frame):
    f = frame[16:96, 16:96, 0]   
    f = f.reshape(5, 16, 5, 16).max(axis=(1, 3))
    f = f.flatten()
    f = f / 255
    return np.expand_dims(f, axis=0)

## Training

#### Test Suite

In [10]:
def test_agent(stochastic=False):
    current_epsilon = agent.epsilon
    agent.epsilon = 0.0
    num_solved = 0
    solved_in_steps = defaultdict(int)

    for t in tqdm(range(100)):    
        random.seed(t)
        sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
        sok.set_maxsteps(20)
        steps = 0

        state = sok.get_image('rgb_array')
        done = False
        while not done:
            steps += 1
            action = agent.act(process_frame(state), stochastic)
            if action < 4:
                action += 1
            else:
                action += 5
            state, reward, done, info = sok.step(action)

        if 3 in sok.room_state:            
            num_solved += 1
            solved_in_steps[steps] += 1
    
    agent.epsilon = current_epsilon
    print("*" * 30)
    print("Stochastic" if stochastic else "Deterministic")
    print("*" * 30)
    print("Solved: %d" % num_solved)
    print("=" * 30)
    print(solved_in_steps)
    print("*" * 30)

In [11]:
max_episodes = 10000
max_steps = 20

def init_sok(r):
    random.seed(r+100)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(max_steps)
    return sok

In [ ]:
agent = SOK_Agent()

steps_per_episode = []

for e in range(max_episodes):
    if e % 100 == 0:
        test_agent(stochastic=False)
        
    print("Episode: %d" % (e+1))
    
    sok = init_sok(e)
    random.seed(e)
    
    state = process_frame(sok.get_image('rgb_array'))
    room_state = sok.room_state.copy() 
    distances = get_distances(room_state)
    
    for step in range(sok.max_steps):
        action = agent.act(state)
        if action < 4:
            next_state, reward, done, _ = sok.step(action+1) 
        else:
            next_state, reward, done, _ = sok.step(action+5)         
        
        next_state = process_frame(next_state)        
        next_room_state = sok.room_state
        
        if not done:
            reward += box2target_change_reward(room_state, next_room_state, distances)
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state.copy() 
        room_state = next_room_state.copy()                
        
        if (step+1) % agent.replay_rate == 0:
            agent.replay()            
        
        if done:   
            steps_per_episode.append(step+1)
            
            if 3 in sok.room_state:                
                print("SOLVED! Episode %d Steps: %d Epsilon %.4f" % (e+1, step+1, agent.epsilon)) 
                agent.copy_to_prioritized_buffer(step+1)
            
            #agent.save("exp1_episode%d.h5" % (e))            
            break


******************************
Deterministic
******************************
Solved: 3
defaultdict(<class 'int'>, {2: 2, 1: 1})
******************************
Episode: 1
Episode: 2
Episode: 3
Episode: 4
Episode: 5
Episode: 6
Episode: 7
Episode: 8
SOLVED! Episode 8 Steps: 6 Epsilon 0.9322
Episode: 9
Episode: 10
Episode: 11
Episode: 12
Episode: 13
Episode: 14
Episode: 15
Episode: 16
Episode: 17
Episode: 18
Episode: 19
Episode: 20
Episode: 21
Episode: 22
Episode: 23
Episode: 24
Episode: 25
Episode: 26
Episode: 27
SOLVED! Episode 27 Steps: 4 Epsilon 0.7783
Episode: 28
Episode: 29
Episode: 30
SOLVED! Episode 30 Steps: 1 Epsilon 0.7629
Episode: 31
Episode: 32
Episode: 33
Episode: 34
Episode: 35
Episode: 36
Episode: 37
Episode: 38
Episode: 39
Episode: 40
Episode: 41
Episode: 42
Episode: 43
Episode: 44
Episode: 45
Episode: 46
Episode: 47
Episode: 48
Episode: 49
Episode: 50
Episode: 51
Episode: 52
Episode: 53
Episode: 54
Episode: 55
Episode: 56
Episode: 57
Episode: 58
Episode: 59
Episode: 60
Ep


******************************
Deterministic
******************************
Solved: 9
defaultdict(<class 'int'>, {1: 6, 2: 1, 3: 2})
******************************
Episode: 101
Episode: 102
Episode: 103
Episode: 104
Episode: 105
Episode: 106
Episode: 107
Episode: 108
Episode: 109
Episode: 110
Episode: 111
Episode: 112
Episode: 113
Episode: 114
Episode: 115
Episode: 116
Episode: 117
Episode: 118
Episode: 119
Episode: 120
Episode: 121
Episode: 122
Episode: 123
Episode: 124
Episode: 125
Episode: 126
SOLVED! Episode 126 Steps: 2 Epsilon 0.3110
Episode: 127
Episode: 128
SOLVED! Episode 128 Steps: 2 Epsilon 0.3079
Episode: 129
SOLVED! Episode 129 Steps: 2 Epsilon 0.3079
Episode: 130
Episode: 131
Episode: 132
Episode: 133
Episode: 134
Episode: 135
Episode: 136
Episode: 137
Episode: 138
Episode: 139
Episode: 140
Episode: 141
Episode: 142
Episode: 143
SOLVED! Episode 143 Steps: 2 Epsilon 0.2988
Episode: 144
Episode: 145
Episode: 146
Episode: 147
Episode: 148
Episode: 149
Episode: 150
Episode: 


******************************
Deterministic
******************************
Solved: 17
defaultdict(<class 'int'>, {1: 9, 2: 4, 3: 4})
******************************
Episode: 201
SOLVED! Episode 201 Steps: 15 Epsilon 0.2988
Episode: 202
Episode: 203
Episode: 204
Episode: 205
Episode: 206
Episode: 207
Episode: 208
Episode: 209
Episode: 210
Episode: 211
Episode: 212
Episode: 213
Episode: 214
Episode: 215
SOLVED! Episode 215 Steps: 20 Epsilon 0.2988
Episode: 216
Episode: 217
Episode: 218
Episode: 219
SOLVED! Episode 219 Steps: 2 Epsilon 0.2988
Episode: 220
Episode: 221
SOLVED! Episode 221 Steps: 6 Epsilon 0.2988
Episode: 222
SOLVED! Episode 222 Steps: 1 Epsilon 0.2988
Episode: 223
Episode: 224
Episode: 225
Episode: 226
Episode: 227
Episode: 228
Episode: 229
Episode: 230
Episode: 231
Episode: 232
Episode: 233
Episode: 234
SOLVED! Episode 234 Steps: 1 Epsilon 0.2988
Episode: 235
Episode: 236
Episode: 237
Episode: 238
SOLVED! Episode 238 Steps: 2 Epsilon 0.2988
Episode: 239
SOLVED! Episode 2


******************************
Deterministic
******************************
Solved: 24
defaultdict(<class 'int'>, {1: 13, 2: 6, 3: 5})
******************************
Episode: 301
SOLVED! Episode 301 Steps: 1 Epsilon 0.2988
Episode: 302
Episode: 303
Episode: 304
Episode: 305
Episode: 306
SOLVED! Episode 306 Steps: 2 Epsilon 0.2988
Episode: 307
Episode: 308
Episode: 309
Episode: 310
Episode: 311
Episode: 312
Episode: 313
Episode: 314
Episode: 315
Episode: 316
SOLVED! Episode 316 Steps: 16 Epsilon 0.2988
Episode: 317
Episode: 318
Episode: 319
Episode: 320
Episode: 321
Episode: 322
Episode: 323
SOLVED! Episode 323 Steps: 14 Epsilon 0.2988
Episode: 324
Episode: 325
SOLVED! Episode 325 Steps: 1 Epsilon 0.2988
Episode: 326
Episode: 327
Episode: 328
SOLVED! Episode 328 Steps: 2 Epsilon 0.2988
Episode: 329
Episode: 330
Episode: 331
Episode: 332
Episode: 333
Episode: 334
Episode: 335
SOLVED! Episode 335 Steps: 1 Epsilon 0.2988
Episode: 336
Episode: 337
Episode: 338
Episode: 339
Episode: 340
Epi


******************************
Deterministic
******************************
Solved: 32
defaultdict(<class 'int'>, {3: 13, 1: 12, 2: 7})
******************************
Episode: 401
Episode: 402
Episode: 403
Episode: 404
SOLVED! Episode 404 Steps: 3 Epsilon 0.2988
Episode: 405
Episode: 406
SOLVED! Episode 406 Steps: 1 Epsilon 0.2988
Episode: 407
Episode: 408
Episode: 409
Episode: 410
Episode: 411
SOLVED! Episode 411 Steps: 2 Epsilon 0.2988
Episode: 412
Episode: 413
Episode: 414
SOLVED! Episode 414 Steps: 3 Epsilon 0.2988
Episode: 415
Episode: 416
Episode: 417
SOLVED! Episode 417 Steps: 3 Epsilon 0.2988
Episode: 418
Episode: 419
SOLVED! Episode 419 Steps: 3 Epsilon 0.2988
Episode: 420
Episode: 421
Episode: 422
Episode: 423
Episode: 424
Episode: 425
Episode: 426
Episode: 427
Episode: 428
SOLVED! Episode 428 Steps: 3 Epsilon 0.2988
Episode: 429
SOLVED! Episode 429 Steps: 2 Epsilon 0.2988
Episode: 430
Episode: 431
Episode: 432
Episode: 433
Episode: 434
Episode: 435
Episode: 436
SOLVED! Epis

...


******************************
Deterministic
******************************
Solved: 37
defaultdict(<class 'int'>, {3: 15, 2: 9, 1: 13})
******************************
Episode: 2201
SOLVED! Episode 2201 Steps: 1 Epsilon 0.2988
Episode: 2202
Episode: 2203
Episode: 2204
Episode: 2205
Episode: 2206
Episode: 2207
SOLVED! Episode 2207 Steps: 3 Epsilon 0.2988
Episode: 2208
SOLVED! Episode 2208 Steps: 3 Epsilon 0.2988
Episode: 2209
Episode: 2210
Episode: 2211
SOLVED! Episode 2211 Steps: 3 Epsilon 0.2988
Episode: 2212
Episode: 2213
Episode: 2214
Episode: 2215
Episode: 2216
Episode: 2217
SOLVED! Episode 2217 Steps: 2 Epsilon 0.2988
Episode: 2218
SOLVED! Episode 2218 Steps: 3 Epsilon 0.2988
Episode: 2219
SOLVED! Episode 2219 Steps: 2 Epsilon 0.2988
Episode: 2220
Episode: 2221
Episode: 2222
Episode: 2223
Episode: 2224
Episode: 2225
Episode: 2226
SOLVED! Episode 2226 Steps: 2 Epsilon 0.2988
Episode: 2227
Episode: 2228
SOLVED! Episode 2228 Steps: 1 Epsilon 0.2988
Episode: 2229
Episode: 2230
SOLVED!


******************************
Deterministic
******************************
Solved: 34
defaultdict(<class 'int'>, {3: 15, 2: 9, 1: 10})
******************************
Episode: 2301
Episode: 2302
Episode: 2303
Episode: 2304
Episode: 2305
Episode: 2306
SOLVED! Episode 2306 Steps: 2 Epsilon 0.2988
Episode: 2307
Episode: 2308
SOLVED! Episode 2308 Steps: 3 Epsilon 0.2988
Episode: 2309
Episode: 2310
Episode: 2311
Episode: 2312
SOLVED! Episode 2312 Steps: 1 Epsilon 0.2988
Episode: 2313
Episode: 2314
Episode: 2315
Episode: 2316
Episode: 2317
SOLVED! Episode 2317 Steps: 1 Epsilon 0.2988
Episode: 2318
Episode: 2319
Episode: 2320
Episode: 2321
SOLVED! Episode 2321 Steps: 3 Epsilon 0.2988
Episode: 2322
SOLVED! Episode 2322 Steps: 3 Epsilon 0.2988
Episode: 2323
Episode: 2324
SOLVED! Episode 2324 Steps: 1 Epsilon 0.2988
Episode: 2325
SOLVED! Episode 2325 Steps: 3 Epsilon 0.2988
Episode: 2326
Episode: 2327
Episode: 2328
Episode: 2329
Episode: 2330
Episode: 2331
Episode: 2332
Episode: 2333
SOLVED! Ep


******************************
Deterministic
******************************
Solved: 37
defaultdict(<class 'int'>, {3: 18, 1: 11, 2: 8})
******************************
Episode: 2401
SOLVED! Episode 2401 Steps: 3 Epsilon 0.2988
Episode: 2402
Episode: 2403
Episode: 2404
Episode: 2405
Episode: 2406
Episode: 2407
Episode: 2408
Episode: 2409
Episode: 2410
Episode: 2411
Episode: 2412
SOLVED! Episode 2412 Steps: 1 Epsilon 0.2988
Episode: 2413
SOLVED! Episode 2413 Steps: 1 Epsilon 0.2988
Episode: 2414
Episode: 2415
Episode: 2416
SOLVED! Episode 2416 Steps: 3 Epsilon 0.2988
Episode: 2417
Episode: 2418
SOLVED! Episode 2418 Steps: 2 Epsilon 0.2988
Episode: 2419
SOLVED! Episode 2419 Steps: 2 Epsilon 0.2988
Episode: 2420
Episode: 2421
SOLVED! Episode 2421 Steps: 2 Epsilon 0.2988
Episode: 2422
Episode: 2423
SOLVED! Episode 2423 Steps: 3 Epsilon 0.2988
Episode: 2424
SOLVED! Episode 2424 Steps: 1 Epsilon 0.2988
Episode: 2425
Episode: 2426
Episode: 2427
SOLVED! Episode 2427 Steps: 2 Epsilon 0.2988
Epis


******************************
Deterministic
******************************
Solved: 34
defaultdict(<class 'int'>, {3: 16, 2: 8, 1: 10})
******************************
Episode: 2501
Episode: 2502
SOLVED! Episode 2502 Steps: 3 Epsilon 0.2988
Episode: 2503
Episode: 2504
Episode: 2505
Episode: 2506
SOLVED! Episode 2506 Steps: 3 Epsilon 0.2988
Episode: 2507
SOLVED! Episode 2507 Steps: 3 Epsilon 0.2988
Episode: 2508
Episode: 2509
SOLVED! Episode 2509 Steps: 3 Epsilon 0.2988
Episode: 2510
Episode: 2511
Episode: 2512
Episode: 2513
SOLVED! Episode 2513 Steps: 3 Epsilon 0.2988
Episode: 2514
Episode: 2515
Episode: 2516
Episode: 2517
SOLVED! Episode 2517 Steps: 3 Epsilon 0.2988
Episode: 2518
Episode: 2519
SOLVED! Episode 2519 Steps: 2 Epsilon 0.2988
Episode: 2520
SOLVED! Episode 2520 Steps: 1 Epsilon 0.2988
Episode: 2521
SOLVED! Episode 2521 Steps: 3 Epsilon 0.2988
Episode: 2522
Episode: 2523
Episode: 2524
Episode: 2525
SOLVED! Episode 2525 Steps: 2 Epsilon 0.2988
Episode: 2526
Episode: 2527
Epis


******************************
Deterministic
******************************
Solved: 30
defaultdict(<class 'int'>, {3: 14, 1: 10, 2: 6})
******************************
Episode: 2601
Episode: 2602
Episode: 2603
Episode: 2604
Episode: 2605
SOLVED! Episode 2605 Steps: 2 Epsilon 0.2988
Episode: 2606
Episode: 2607
SOLVED! Episode 2607 Steps: 3 Epsilon 0.2988
Episode: 2608
SOLVED! Episode 2608 Steps: 1 Epsilon 0.2988
Episode: 2609
SOLVED! Episode 2609 Steps: 3 Epsilon 0.2988
Episode: 2610
SOLVED! Episode 2610 Steps: 1 Epsilon 0.2988
Episode: 2611
SOLVED! Episode 2611 Steps: 3 Epsilon 0.2988
Episode: 2612
Episode: 2613
